## 处理mol2文件

In [6]:


mol2file = "./input/4ek3_lig"
pdbfile = "./input/4ek3_pro_fix"

### 得到小分子的bcc电荷以及pdb复合物

In [7]:
# get bcc charges
os.system("~/miniconda3/envs/ambertools/bin/antechamber -i %s.mol2 -fi mol2 -o %s.mol2 -fo mol2 -c bcc -nc 0" % (mol2file, mol2file))

256

In [8]:
os.system("cat %s.pdb %s_0001.pdb > complex.pdb" % (pdbfile, mol2file))

0

## GALigandDock Protocol

In [9]:
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib
%matplotlib inline
import os
import pandas as pd
import pyrosetta
import pyrosetta.distributed
import pyrosetta.distributed.io as io
import pyrosetta.distributed.viewer as viewer
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import seaborn
seaborn.set()
import sys

!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta

In [10]:
pdb_filename = "./input/complex.pdb"
ligand_params = f"{mol2file}.params"
flags = f"""
-ignore_unrecognized_res 1
-extra_res_fa {ligand_params}
-beta_cart
-out:level 200
"""
pyrosetta.distributed.init(flags)
pose_obj = io.pose_from_file(filename=pdb_filename)

INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CG  on residue GLU 73
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CD  on residue GLU 73
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OE1 on residue GLU 73
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OE2 on residue GLU 73
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CG  on residue ASN 74
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OD1 on residue ASN 74
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  ND2 on residue ASN 74
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CG  on residue LYS 75
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CD  on residue LYS 75
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heav

RuntimeError: Trying to save an unregistered polymorphic type (core::scoring::GenBondedExclInfo).
Make sure your type is registered with CEREAL_REGISTER_TYPE and that the archive you are using was included (and registered with CEREAL_REGISTER_ARCHIVE) prior to calling CEREAL_REGISTER_TYPE.
If your type is already registered and you still see this error, you may need to use CEREAL_REGISTER_DYNAMIC_INIT.

In [ ]:
xml = f"""
<ROSETTASCRIPTS>
  <SCOREFXNS>
    <ScoreFunction name="fa_standard" weights="beta_cart.wts"/>
  </SCOREFXNS>
  <MOVERS>
    <GALigandDock name="dock"
                  scorefxn="fa_standard"
                  scorefxn_relax="fa_standard"
                  grid_step="0.25"
                  padding="5.0"
                  hashsize="8.0"
                  subhash="3"
                  nativepdb="{pdb_filename}"
                  final_exact_minimize="sc"
                  random_oversample="10"
                  rotprob="0.9"
                  rotEcut="100"
                  sidechains="auto"
                  initial_pool="{pdb_filename}">
      <Stage repeats="10" npool="50" pmut="0.2" smoothing="0.375" rmsdthreshold="2.5" maxiter="50" pack_cycles="100" ramp_schedule="0.1,1.0"/>
      <Stage repeats="10" npool="50" pmut="0.2" smoothing="0.375" rmsdthreshold="1.5" maxiter="50" pack_cycles="100" ramp_schedule="0.1,1.0"/>
    </GALigandDock>
  </MOVERS>
  <PROTOCOLS>
    <Add mover="dock"/>
  </PROTOCOLS>
</ROSETTASCRIPTS>
"""
xml_obj = rosetta_scripts.MultioutputRosettaScriptsTask(xml)
xml_obj.setup()

INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU 73
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU 73
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU 73
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU 73
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ASN 74
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OD1 on residue ASN 74
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  ND2 on residue ASN 74
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 75
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 75
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 75
INFO:roset

In [ ]:
if not os.getenv("DEBUG"):
    %time results = list(xml_obj(pose_obj))

INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU 73
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU 73
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU 73
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU 73
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ASN 74
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OD1 on residue ASN 74
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  ND2 on residue ASN 74
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 75
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 75
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 75
INFO:roset

RuntimeError: 

File: /home/benchmark/rosetta/source/src/protocols/ligand_docking/GALigandDock/GALigandDock.cc:2566
[ ERROR ] UtilityExitException
ERROR: error!  pharmacophore docking requires reference_pool to be as 'INPUT'!

